In [3]:
from gutenberg.acquire import load_etext
from gutenberg.cleanup import strip_headers
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import numpy as np
import scipy.io as sio

In [4]:
 # Loads Moby Dick
mobydick = strip_headers(load_etext(2701)).strip()
text = [mobydick.split('ETYMOLOGY')[-1]]

In [5]:
# Loads Pride and Prejudice
pride = strip_headers(load_etext(1342)).strip()
text.append(pride.split('Austen')[-1])

In [6]:
# Loads Sense and Sensibility
sense = strip_headers(load_etext(161)).strip()
text.append(sense.split('\n(1811)\n\n')[-1])

In [7]:
# Loads Huck Finn
huckfinn = strip_headers(load_etext(76)).strip()
text.append(huckfinn.split('\nEXPLANATORY\n\n')[-1])

In [8]:
# Loads tom sawyer
toms = strip_headers(load_etext(74)).strip()
text.append(toms.split('\nPREFACE\n\n')[-1])

In [9]:
# Loads Twenty Thousand Leagues
leagues20k = strip_headers(load_etext(164)).strip()
text.append(leagues20k.split(' VERNE\n\n')[-1])

In [10]:
# Loads Ulysses
ulysses = strip_headers(load_etext(4300)).strip()
text.append(ulysses.split('\n— I —\n')[-1])

In [11]:
titles = ['Moby Dick','Pride and Prejudice','Huck Finn','Tom Sawyer','Sense and Sensibility','20K Leagues','Ulysses']

In [12]:
print(ulysses.split('\n— I —\n')[-1][:1000])



[ 1 ]

Stately, plump Buck Mulligan came from the stairhead, bearing a bowl of
lather on which a mirror and a razor lay crossed. A yellow
dressinggown, ungirdled, was sustained gently behind him on the mild
morning air. He held the bowl aloft and intoned:

—_Introibo ad altare Dei_.

Halted, he peered down the dark winding stairs and called out coarsely:

—Come up, Kinch! Come up, you fearful jesuit!

Solemnly he came forward and mounted the round gunrest. He faced about
and blessed gravely thrice the tower, the surrounding land and the
awaking mountains. Then, catching sight of Stephen Dedalus, he bent
towards him and made rapid crosses in the air, gurgling in his throat
and shaking his head. Stephen Dedalus, displeased and sleepy, leaned
his arms on the top of the staircase and looked coldly at the shaking
gurgling face that blessed him, equine in its length, and at the light
untonsured hair, grained and hued like pale oak.

Buck Mulligan peeped an instant under the mirror and then

In [13]:
chapters = []
for i in range(len(text)):
    chapters.append(text[i].split('CHAPTER '))
chapters[1] = text[1].split('Chapter ') # fix
chapters[6] = text[6].split(' ]\n\n') # fix
allchaps = [y for x in chapters for y in x]

In [14]:
print(chapters[6][-1][:200])

Yes because he never did a thing like that before as ask to get his
breakfast in bed with a couple of eggs since the _City Arms_ hotel when
he used to be pretending to be laid up with a sick voice doi


In [15]:
chapix = [len(x) for x in chapters]

In [16]:
def textcounter(chapters,n_features=1000):
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                       max_features=n_features,
                                       stop_words='english')
    freq = tfidf_vectorizer.fit_transform(chapters)
    words = tfidf_vectorizer.get_feature_names()
    return freq, words

In [17]:
freq, words = textcounter(allchaps,50)

In [18]:
words[freq.sum(0).argmax()]

'whale'

In [19]:
sio.savemat('/Users/zaharia/books.mat',{'freq':freq.todense(),'words':np.array(words,dtype=np.object),\
                                        'chapix':chapix,'titles':np.array(titles,dtype=np.object)})

In [38]:
sepfreq = []
sepwords = []
for i in range(len(titles)):
    tmpfreq, tmpwords = textcounter(chapters[i],10)
    sepfreq.append(tmpfreq)
    sepwords.append(tmpwords)

In [39]:
sepwords

[['ahab', 'head', 'like', 'man', 'old', 'sea', 'ship', 'time', 'whale', 'ye'],
 ['bennet',
  'bingley',
  'darcy',
  'did',
  'jane',
  'know',
  'miss',
  'sister',
  'soon',
  'think'],
 ['dashwood',
  'did',
  'edward',
  'jennings',
  'know',
  'miss',
  'mother',
  'said',
  'sister',
  'willoughby'],
 ['ain', 'don', 'en', 'going', 'jim', 'like', 'll', 'man', 'old', 'tom'],
 ['boy', 'boys', 'did', 'don', 'huck', 'joe', 'just', 'little', 'll', 'said'],
 ['captain',
  'conseil',
  'land',
  'like',
  'nautilus',
  'ned',
  'nemo',
  'said',
  'sir',
  'water'],
 ['bloom',
  'did',
  'like',
  'man',
  'mr',
  'old',
  'said',
  'says',
  'stephen',
  'time']]

In [40]:
sepwords = [y for x in sepwords for y in x]
sepfreq = [y.todense() for x in sepfreq for y in x]

In [32]:
sepfreq

[matrix([[0.        , 0.0699783 , 0.01764176, 0.02923735, 0.17992809,
          0.11194029, 0.08615676, 0.03673482, 0.02366785, 0.05034827,
          0.08131771, 0.23295578, 0.12237243, 0.04035396, 0.02606566,
          0.02588398, 0.84077474, 0.38004374, 0.01665641, 0.08196351]]),
 matrix([[0.        , 0.        , 0.13707633, 0.28396753, 0.14979001,
          0.21744383, 0.25103907, 0.09514312, 0.13792428, 0.29340433,
          0.        , 0.65363458, 0.15847218, 0.        , 0.30379504,
          0.3016775 , 0.13423592, 0.        , 0.        , 0.        ]]),
 matrix([[0.        , 0.        , 0.        , 0.40163052, 0.17654672,
          0.        , 0.14794085, 0.        , 0.        , 0.69162915,
          0.22341065, 0.        , 0.        , 0.41575374, 0.08951534,
          0.17778279, 0.07910711, 0.1134918 , 0.11440373, 0.11259244]]),
 matrix([[0.        , 0.02943713, 0.        , 0.17218614, 0.12975208,
          0.42380012, 0.30806365, 0.2266426 , 0.33850873, 0.31769325,
          0

In [41]:
sio.savemat('/Users/zaharia/books.mat',{'freq':sepfreq,'words':np.array(sepwords,dtype=np.object),\
                                        'chapix':chapix,'titles':np.array(titles,dtype=np.object)})

In [37]:
len(allchaps)

409